# 1. Feature Engineering

In [30]:
import numpy as np
import pandas as pd
import pandas_profiling
import datetime as dt
from tqdm import tqdm
import sys

### 1-1. 주식 데이터 관련 feature 추가

In [2]:
train=pd.read_csv("trade_train.csv")
train

,Unnamed: 0,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
0,0,201907,MAD01,288,A000660,Y,Y,7,17,19,234,74800,78500
1,1,201907,MAD01,288,A001820,Y,Y,3,4,533,154,47385,46700
2,2,201907,MAD01,288,A004020,Y,Y,3,3,409,528,41411,40000
3,3,201907,MAD01,288,A005380,Y,Y,4,3,323,451,140602,140628
4,4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30195,30195,202006,MAD48,163,A207940,Y,Y,3,4,3,3,835000,797000
30196,30196,202006,MAD48,163,A272210,Y,Y,3,3,90,42,9900,8870
30197,30197,202006,MAD48,163,A285130,Y,Y,6,4,271,162,109100,98000
30198,30198,202006,MAD48,163,A316140,Y,Y,5,4,386,735,10050,9920


In [3]:
stocks = pd.read_csv("stocks.csv")
stocks

,index,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,종목시가,종목고가,종목저가,종목종가,거래량,거래금액_만원단위
0,0,20190701,A000020,동화약품,N,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,9680,9840,9680,9710,31341,30581
1,1,20190701,A000080,하이트진로,N,코스피,제조업,음료 제조업,알코올음료 제조업,20550,20550,20200,20300,127186,258535
2,2,20190701,A000100,유한양행,Y,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,259000,267500,250000,251500,395757,10139357
3,3,20190701,A000120,CJ대한통운,N,코스피,운수 및 창고업,육상운송 및 파이프라인 운송업,도로 화물 운송업,136500,137000,133000,136000,62513,846100
4,4,20190701,A000140,하이트진로홀딩스,Y,코스피,금융 및 보험업,금융업,기타 금융업,9060,9190,8910,9000,7661,6896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286056,286056,20200728,A336370,두산솔루스,N,코스피,제조업,통신장비 제조업,전자부품 제조업,37050,39150,37000,38150,1764085,6755086
286057,286057,20200728,A33637K,두산솔루스1우,N,코스피,제조업,통신장비 제조업,전자부품 제조업,18100,19350,17950,18250,302925,565624
286058,286058,20200728,A33637L,두산솔루스2우B,N,코스피,제조업,통신장비 제조업,전자부품 제조업,37000,38650,36500,37700,59460,224214
286059,286059,20200728,A344820,케이씨씨글라스,N,코스피,제조업,비금속 광물제품 제조업,유리 및 유리제품 제조업,33500,33500,32250,32400,56962,186549


- 일별 데이터 추가

In [4]:
stocks['당일등락률']=((stocks['종목종가']-stocks['종목시가'])/stocks['종목시가'])*100

In [5]:
list = pd.unique(stocks['종목번호'])
df2 = pd.DataFrame(columns = stocks.columns)
for i in list:
    df = stocks[stocks['종목번호']==i]
    df['전일종가'] = np.nan
    for j in range(len(df)):
        if j == 0:
            continue
        else:
            df['전일종가'].iloc[j] = df['종목종가'].iloc[j-1]
    df1 = df
    frames = [df2, df1]
    df2 = pd.concat(frames)
stocks = pd.merge(stocks, df2, how = 'outer')

c:\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\python\python36\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remo

In [9]:
stocks['전일비'] = stocks['종목종가'] - stocks['전일종가']
stocks['등락률'] = stocks['전일비']/stocks['전일종가']
stocks['종목시가-전일종가'] = stocks['종목시가'] - stocks['전일종가']
stocks['당일등락량']=abs(stocks['종목고가']-stocks['종목저가'])
stocks['증가율']=""
for i in range(len(stocks)):
    if stocks['종목저가'][i]==0:
        stocks['증가율'][i]=0
    else:
        stocks['증가율'][i]=(stocks['종목고가'][i]-stocks['종목저가'][i])/stocks['종목저가'][i]

c:\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


- 월별 데이터 추가

In [11]:
stocks['기준일자'] = pd.to_datetime(stocks['기준일자'], format='%Y%m%d')
stocks['기준년월'] = stocks['기준일자'].dt.to_period('M')
stocks.head()

,index,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,종목시가,...,거래량,거래금액_만원단위,당일등락률,전일종가,전일비,등락률,종목시가-전일종가,당일등락량,증가율,기준년월
0,0,2019-07-01,A000020,동화약품,N,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,9680,...,31341,30581,0.309917,NaN,NaN,NaN,NaN,160,0.0165289,2019-07
1,1,2019-07-01,A000080,하이트진로,N,코스피,제조업,음료 제조업,알코올음료 제조업,20550,...,127186,258535,-1.216545,NaN,NaN,NaN,NaN,350,0.0173267,2019-07
2,2,2019-07-01,A000100,유한양행,Y,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,259000,...,395757,10139357,-2.895753,NaN,NaN,NaN,NaN,17500,0.07,2019-07
3,3,2019-07-01,A000120,CJ대한통운,N,코스피,운수 및 창고업,육상운송 및 파이프라인 운송업,도로 화물 운송업,136500,...,62513,846100,-0.366300,NaN,NaN,NaN,NaN,4000,0.0300752,2019-07
4,4,2019-07-01,A000140,하이트진로홀딩스,Y,코스피,금융 및 보험업,금융업,기타 금융업,9060,...,7661,6896,-0.662252,NaN,NaN,NaN,NaN,280,0.0314254,2019-07


In [12]:
stocks['증가율'] = pd.to_numeric(stocks['증가율'])
stocks['당일등락량'] = pd.to_numeric(stocks['당일등락량'])

In [13]:
tmp1 = stocks.groupby(['종목번호', '기준년월', '종목명'])['거래량'].sum().reset_index()
tmp2 = stocks.groupby(['종목번호', '기준년월', '종목명'])['종목고가'].max().reset_index()
stocks2 = pd.merge(tmp1, tmp2, how = 'outer')

tmp3 = stocks.groupby(['종목번호', '기준년월', '종목명'])['종목저가'].min().reset_index()
stocks2 = pd.merge(stocks2, tmp3, how = 'outer')

tmp4 = stocks.groupby(['종목번호', '기준년월', '종목명'])['증가율'].max().reset_index()
stocks2 = pd.merge(stocks2, tmp4, how = 'outer')
stocks2.rename(columns = {'증가율' : '최고증가율'}, inplace = True)

tmp5 = stocks.groupby(['종목번호', '기준년월', '종목명'])['증가율'].min().reset_index()
stocks2 = pd.merge(stocks2, tmp5, how = 'outer')
stocks2.rename(columns = {'증가율' : '최저증가율'}, inplace = True)

tmp6 = stocks.groupby(['종목번호', '기준년월', '종목명'])['증가율'].mean().reset_index()
stocks2 = pd.merge(stocks2, tmp6, how = 'outer')
stocks2.rename(columns = {'증가율' : '평균증가율'}, inplace = True)

tmp7 = stocks.groupby(['종목번호', '기준년월', '종목명'])['당일등락량'].mean().reset_index()
stocks2 = pd.merge(stocks2, tmp7, how = 'outer')
stocks2.rename(columns = {'증가율' : '평균증감량'}, inplace = True)



stocks2

,종목번호,기준년월,종목명,거래량,종목고가,종목저가,최고증가율,최저증가율,평균증가율,당일등락량
0,A000020,2019-07,동화약품,1197340,9960,8700,0.059574,0.011340,0.029032,269.565217
1,A000020,2019-08,동화약품,3304334,9500,7500,0.153333,0.016166,0.055384,460.952381
2,A000020,2019-09,동화약품,2025229,8680,7860,0.078261,0.011222,0.029633,237.368421
3,A000020,2019-10,동화약품,2634273,8440,7500,0.058065,0.009009,0.030816,239.047619
4,A000020,2019-11,동화약품,2455546,8520,7710,0.062344,0.012225,0.026717,215.714286
...,...,...,...,...,...,...,...,...,...,...
13907,A344820,2020-05,케이씨씨글라스,4841887,34600,27800,0.127036,0.024433,0.052820,1550.000000
13908,A344820,2020-06,케이씨씨글라스,5436720,38150,30450,0.127419,0.029730,0.058690,1922.727273
13909,A344820,2020-07,케이씨씨글라스,2170002,35900,31200,0.064103,0.015699,0.036560,1192.500000
13910,A353810,2020-06,이지바이오,10126607,71100,0,0.299145,0.000000,0.126152,5386.111111


In [14]:
tmp = stocks[stocks['종목시가']==0]
stop_list = tmp['종목번호'].unique()

In [15]:
stocks2['거래중지이력']=""
for i in range(len(stocks2)):
    if stocks2['종목번호'][i] in stop_list:
        stocks2['거래중지이력'][i]=1
    else:
        stocks2['거래중지이력'][i]=0

c:\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
stocks2

,종목번호,기준년월,종목명,거래량,종목고가,종목저가,최고증가율,최저증가율,평균증가율,당일등락량,거래중지이력
0,A000020,2019-07,동화약품,1197340,9960,8700,0.059574,0.011340,0.029032,269.565217,0
1,A000020,2019-08,동화약품,3304334,9500,7500,0.153333,0.016166,0.055384,460.952381,0
2,A000020,2019-09,동화약품,2025229,8680,7860,0.078261,0.011222,0.029633,237.368421,0
3,A000020,2019-10,동화약품,2634273,8440,7500,0.058065,0.009009,0.030816,239.047619,0
4,A000020,2019-11,동화약품,2455546,8520,7710,0.062344,0.012225,0.026717,215.714286,0
...,...,...,...,...,...,...,...,...,...,...,...
13907,A344820,2020-05,케이씨씨글라스,4841887,34600,27800,0.127036,0.024433,0.052820,1550.000000,0
13908,A344820,2020-06,케이씨씨글라스,5436720,38150,30450,0.127419,0.029730,0.058690,1922.727273,0
13909,A344820,2020-07,케이씨씨글라스,2170002,35900,31200,0.064103,0.015699,0.036560,1192.500000,0
13910,A353810,2020-06,이지바이오,10126607,71100,0,0.299145,0.000000,0.126152,5386.111111,1


### 1-2. 시가총액 및 외국인매수 정보 크롤링 및 추가

- 2019년 시가총액 정보 (출처: https://github.com/FinanceData/marcap)

In [3]:
df_2019 = pd.read_csv("marcap-2019.csv")
df_2019_col = df_2019[['Date', 'Name', 'Marcap', 'ForeignRatio', 'Rank']]

In [42]:
df_2019_col['Date'] = pd.to_datetime(df_2019_col['Date'], format='%Y-%m-%d')
df_2019_col['기준년월'] = df_2019_col['Date'].dt.to_period('M')
df_2019_col

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,Name,Marcap,ForeignRatio,Rank,기준년월
0,2019-01-02,삼성전자,231329073812500,55.61,1.0,2019-01
1,2019-01-02,SK하이닉스,44116943319000,48.39,2.0,2019-01
2,2019-01-02,셀트리온,26910340528500,19.71,3.0,2019-01
3,2019-01-02,삼성전자우,26003219720000,92.21,4.0,2019-01
4,2019-01-02,삼성바이오로직스,24745710000000,8.62,5.0,2019-01
...,...,...,...,...,...,...
590006,2019-12-30,씨앤에스링크,2701731600,0.00,2471.0,2019-12
590007,2019-12-30,엘리비젼,2587200000,0.00,2472.0,2019-12
590008,2019-12-30,동양3우B,2552590900,0.00,2473.0,2019-12
590009,2019-12-30,로보쓰리,2224028850,0.65,2474.0,2019-12


In [42]:
df_2019_col['Date'] = pd.to_datetime(df_2019_col['Date'], format='%Y-%m-%d')
df_2019_col['기준년월'] = df_2019_col['Date'].dt.to_period('M')
df_2019_col

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,Name,Marcap,ForeignRatio,Rank,기준년월
0,2019-01-02,삼성전자,231329073812500,55.61,1.0,2019-01
1,2019-01-02,SK하이닉스,44116943319000,48.39,2.0,2019-01
2,2019-01-02,셀트리온,26910340528500,19.71,3.0,2019-01
3,2019-01-02,삼성전자우,26003219720000,92.21,4.0,2019-01
4,2019-01-02,삼성바이오로직스,24745710000000,8.62,5.0,2019-01
...,...,...,...,...,...,...
590006,2019-12-30,씨앤에스링크,2701731600,0.00,2471.0,2019-12
590007,2019-12-30,엘리비젼,2587200000,0.00,2472.0,2019-12
590008,2019-12-30,동양3우B,2552590900,0.00,2473.0,2019-12
590009,2019-12-30,로보쓰리,2224028850,0.65,2474.0,2019-12


- 2020년 3월까지 시가총액 정보 (출처: https://github.com/FinanceData/marcap)

In [11]:
df_2020_0103 = pd.read_csv("marcap-2020.csv")
df_2020_0103_col = df_2020_0103[['Date', 'Name', 'Marcap', 'ForeignRatio', 'Rank']]

In [14]:
df_2020_0103_col['Date'] = pd.to_datetime(df_2020_0103_col['Date'], format='%Y-%m-%d')
df_2020_0103_col['기준년월'] = df_2020_0103_col['Date'].dt.to_period('M')
df_2020_0103_col['기준년월']=df_2020_0103_col['기준년월'].astype(str)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [15]:
df_2020_0103_col.rename(columns = {'Name' : '종목명'}, inplace = True)
df_2020_0103_col.rename(columns = {'Marcap' : '시가총액'}, inplace = True)
df_2020_0103_col.rename(columns = {'ForeignRatio' : '외국인 지분율(%)'}, inplace = True)
df_2020_0103_col.rename(columns = {'Rank' : '순위'}, inplace = True)

c:\python\python36\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


- 2020년 4~7월 시가총액 정보 크롤링 (출처: https://www.krx.co.kr)

In [ ]:
from selenium import webdriver
import datetime as dt
import time

In [32]:
driver = webdriver.Chrome('C:/chromedriver.exe')

In [42]:
dt_index = pd.date_range(start='20200401', end='20200731')
dt_list = dt_index.strftime("%Y%m%d").tolist()

for i in dt_list:
    driver.get('https://marketdata.krx.co.kr/mdi#document=040402')
    
    for j in range(2):
        if j==0:
            kospi_checkbox = driver.find_element_by_xpath('//*[@id="market_gubun1a87ff679a2f3e71d9181a67b7542122c"]')
            kospi_checkbox.click()
        else:
            kosdaq_checkbox = driver.find_element_by_xpath('//*[@id="market_gubun2a87ff679a2f3e71d9181a67b7542122c"]')
            kosdaq_checkbox.click()
        time.sleep(0.5)
        date = driver.find_element_by_xpath('//*[@id="schdate8f14e45fceea167a5a36dedd4bea2543"]')
        date.click()
        date.clear()
        date.send_keys(i)
        time.sleep(0.5)


        view = driver.find_element_by_xpath('//*[@id="btnidc81e728d9d4c2f636f067f89cc14862c"]')
        view.click()
        time.sleep(2)
        
        down = driver.find_element_by_xpath('//*[@onclick="$.down(this,\'csv\',\'formc81e728d9d4c2f636f067f89cc14862c\');return false;"]')
        down.click()
        time.sleep(1)


In [17]:
tmp = pd.read_csv("data (1).csv")
list = pd.date_range(start="2020-04-01",end="2020-07-31").tolist()
df_2020_0407 = pd.DataFrame(columns = tmp.columns)
for i in range(122):
    df = pd.read_csv("data ("+str(2*(i+1)-1)+").csv")
    df['Date']= list[i]
    df_2020_0407 = pd.concat([df_2020_0407, df])
    
for i in range(122):
    df = pd.read_csv("data ("+str(2*(i+1))+").csv")
    df['Date']= list[i]
    df_2020_0407 = pd.concat([df_2020_0407, df])

In [18]:
df_2020_0407

,순위,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수,외국인 보유주식수,외국인 지분율(%),총카운트,Date
0,1,005930,삼성전자,"45,800","-1,950",-4.1,"27,259,532","1,282,482,561,850","47,450","47,900","45,800","273,416,040,790,000",24.11,"5,969,782,550","3,282,913,127",54.99,915.0,2020-04-01
1,2,000660,SK하이닉스,"78,400","-4,900",-5.9,"6,845,675","552,936,384,600","82,100","83,200","78,400","57,075,385,416,000",5.03,"728,002,365","357,900,304",49.16,NaN,2020-04-01
2,3,005935,삼성전자우,"39,000","-1,100",-2.7,"2,688,956","106,901,374,900","40,100","40,550","39,000","32,092,581,300,000",2.83,"822,886,700","729,713,444",88.68,NaN,2020-04-01
3,4,207940,삼성바이오로직스,"452,500","-29,500",-6.1,"313,016","147,210,148,000","481,000","486,000","450,500","29,939,662,500,000",2.64,"66,165,000","6,883,841",10.40,NaN,2020-04-01
4,5,035420,NAVER,"163,000","-7,000",-4.1,"720,283","119,915,823,500","166,500","171,000","163,000","26,774,933,385,000",2.36,"164,263,395","95,606,112",58.20,NaN,2020-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,1421,021045,대호피앤씨우,"1,480",5,0.3,"48,085","70,822,525","1,480","1,485","1,465","6,278,849,680",0.00,"4,242,466","90,096",2.12,NaN,2020-07-31
1421,1422,347140,케이프이에스제4호,"2,030",0,0.0,"5,879","11,928,985","2,035","2,035","2,020","6,108,270,000",0.00,"3,009,000",123,0.00,NaN,2020-07-31
1422,1423,08537M,루트로닉3우C,"8,430",-460,-5.2,"1,878","15,882,470","8,410","8,880","8,410","6,100,681,410",0.00,"723,687",687,0.09,NaN,2020-07-31
1423,1424,158310,스타모빌리티,505,0,0.0,0,0,0,0,0,"6,096,179,210",0.00,"12,071,642","533,137",4.42,NaN,2020-07-31


In [20]:
df_2020_0407_col = df_2020_0407[['Date', '종목명', '시가총액', '외국인 지분율(%)', '순위']]
df_2020_0407_col['Date'] = pd.to_datetime(df_2020_0407_col['Date'], format='%Y-%m-%d')
df_2020_0407_col['기준년월'] = df_2020_0407_col['Date'].dt.to_period('M')
df_2020_0407_col['기준년월']=df_2020_0407_col['기준년월'].astype(str)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [23]:
frame = [df_2019_col, df_2020_0103_col, df_2020_0407_col]
merged = pd.concat(frame)

In [25]:
df4 = merged.groupby(['기준년월', '종목명'])['외국인 지분율(%)'].max().reset_index()
df4.rename(columns = {'외국인 지분율(%)' : '최대 외국인 지분율'}, inplace = True)
df5 = merged.groupby(['기준년월', '종목명'])['외국인 지분율(%)'].min().reset_index()
df5.rename(columns = {'외국인 지분율(%)' : '최소 외국인 지분율'}, inplace = True)
df5 = pd.merge(df4, df5, on = ['기준년월', '종목명'], how = 'outer')
df5['외국인 지분율 최대변동비율'] = df5['최대 외국인 지분율'] - df5['최소 외국인 지분율']
del df5['최대 외국인 지분율']
del df5['최소 외국인 지분율']

In [29]:
merged['순위'] = merged['순위'].astype(np.float)

df1 = merged.groupby(['기준년월', '종목명'])['시가총액'].max().reset_index()
stocks3 = pd.merge(stocks2, df1, on = ['기준년월', '종목명'], how = 'left')

df2 = merged.groupby(['기준년월', '종목명'])['외국인 지분율(%)'].mean().reset_index()
stocks3 = pd.merge(stocks3, df2, on = ['기준년월', '종목명'], how = 'left')

df3 = merged.groupby(['기준년월', '종목명'])['순위'].mean().reset_index()
stocks3 = pd.merge(stocks3, df3, on = ['기준년월', '종목명'], how = 'left')

df4 = merged.groupby(['기준년월', '종목명'])['외국인 지분율(%)'].max().reset_index()
df4.rename(columns = {'외국인 지분율(%)' : '최대 외국인 지분율'}, inplace = True)
df5 = merged.groupby(['기준년월', '종목명'])['외국인 지분율(%)'].min().reset_index()
df5.rename(columns = {'외국인 지분율(%)' : '최소 외국인 지분율'}, inplace = True)
df5 = pd.merge(df4, df5, on = ['기준년월', '종목명'], how = 'outer')
df5['외국인 지분율 최대변동비율'] = (df5['최대 외국인 지분율'] - df5['최소 외국인 지분율'])/df5['최소 외국인 지분율']
del df5['최대 외국인 지분율']
del df5['최소 외국인 지분율']

stocks3 = pd.merge(stocks2, df5, on = ['기준년월', '종목명'], how = 'left')

In [47]:
stocks3.rename(columns = {'외국인 지분율(%)' : '외국인 지분율'}, inplace = True)
stocks3['외국인 지분율'] = stocks3['외국인 지분율']/100
stocks3

,종목번호,기준년월,종목명,거래량,종목고가,종목저가,최고증가율,최저증가율,평균증가율,당일등락량,거래중지이력,시가총액,외국인 지분율,순위,외국인 지분율 최대변동비율
0,A000020,2019-07,동화약품,1197340,9960,8700,0.059574,0.011340,0.029032,269.565217,0,273728406000,0.062922,567.043478,0.072488
1,A000020,2019-08,동화약품,3304334,9500,7500,0.153333,0.016166,0.055384,460.952381,0,262555818000,0.056362,560.571429,0.263048
2,A000020,2019-09,동화약품,2025229,8680,7860,0.078261,0.011222,0.029633,237.368421,0,232948459800,0.048784,624.578947,0.069915
3,A000020,2019-10,동화약품,2634273,8440,7500,0.058065,0.009009,0.030816,239.047619,0,226803536400,0.050081,642.333333,0.044898
4,A000020,2019-11,동화약품,2455546,8520,7710,0.062344,0.012225,0.026717,215.714286,0,233786403900,0.048981,633.238095,0.122318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13907,A344820,2020-05,케이씨씨글라스,4841887,34600,27800,0.127036,0.024433,0.052820,1550.000000,0,"268,459,251,500",0.070137,366.684211,0.242009
13908,A344820,2020-06,케이씨씨글라스,5436720,38150,30450,0.127419,0.029730,0.058690,1922.727273,0,"316,055,448,500",0.068173,343.500000,0.115920
13909,A344820,2020-07,케이씨씨글라스,2170002,35900,31200,0.064103,0.015699,0.036560,1192.500000,0,"295,179,923,500",0.065448,352.565217,0.030864
13910,A353810,2020-06,이지바이오,10126607,71100,0,0.299145,0.000000,0.126152,5386.111111,1,"98,900,986,000",0.057345,577.500000,0.265504


In [48]:
stocks3.to_csv('data/stock_factor3.csv', index=False)

### 1-3. 지수평활법 예측 feature 생성

In [1]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing as ES
from tqdm import trange
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
df = pd.read_csv('data/trade_train.csv', index_col=0)
df = df[['그룹번호', '종목번호', '매수고객수']]

In [37]:
def exp_smoothing(window=6, trend='add'):
    first=True
    # Repeat for all guest groups
    for i in range(1,49):
        mad = df[(df['그룹번호']=='MAD'+str(i).zfill(2))][['그룹번호', '종목번호', '매수고객수']]
        code_list = mad[mad['매수고객수']!=0]['종목번호'].unique()
        
        # Repeat for all stock codes
        for code in tqdm(code_list):
            temp = mad[mad['종목번호']==code].copy()
            temp['매수고객수_ES'+str(window)] = 0
            l_w = 0
            length = len(df.index.unique())
            
            for j in range(window, length-1):
                model = ES(temp['매수고객수'][l_w : l_w + window], trend='add').fit()
                pred = model.predict()[0]
                temp.loc[temp.index[j], '매수고객수_ES'+str(window)] = pred
                l_w += 1
            
            if first==True:
                result = temp
            else:
                result = pd.concat([result, temp], axis=0)
            first=False

    return result

- 기간 3개월로 예측

In [38]:
window_size=3
result = exp_smoothing(window=window_size)

100%|██████████| 80/80 [00:08<00:00,  9.19it/s]


In [41]:
result.to_csv('data/stock_factor_ES'+str(window_size)+'.csv')

- 기간 6개월로 예측

In [ ]:
window_size=6
result = exp_smoothing(window=window_size)

In [43]:
result.to_csv('data/stock_factor_ES'+str(window_size)+'.csv')

### 1-4. Train data와 Stock data, 생성된 feature 병합

In [2]:
df_train = pd.read_csv('data/trade_train.csv', index_col=0)
df_stock = pd.read_csv('data/stocks.csv', index_col=0)
factor = pd.read_csv('data/stock_factor3.csv')
factor['기준년월'] = pd.to_datetime(factor['기준년월'], format='%Y-%m')
factor['기준년월'] = factor['기준년월'].dt.to_period('M')

- train data와 stock data, 1-1, 1-2 에서 생성한 feature 병합

In [3]:
def preprocess(customer: int):
    # Check group number
    temp = df_train[df_train['그룹번호']=='MAD'+str(customer).zfill(2)].sort_values(['기준년월', '매수고객수'], ascending=[True, False])
    temp.reset_index(inplace=True, drop=True)
    
    ## Make new dataframe for prediction
    
    # Make columns with train data
    df = pd.DataFrame((list(df_train['기준년월'].unique())+[202007])*1087, columns=['기준년월']).sort_values('기준년월')
    df['그룹번호'] = 'MAD'+str(customer).zfill(2)
    df.reset_index(inplace=True, drop=True)
    df['종목번호'] = list(df_stock['종목번호'].unique())*13
    df = pd.concat([df, pd.DataFrame(np.zeros((1087*13,6)), columns=['매수고객수' , 
                                                                    '전월_매도고객수', 
                                                                    '전월_평균매수수량', 
                                                                    '전월_평균매도수량', 
                                                                    '전월_매수가격_중앙값', 
                                                                    '전월_매도가격_중앙값'])], axis=1)
    
    # Fill up the data
    for i in range(len(temp)):

        # Fill up 매수고객수   
        index = df[(df['종목번호']==temp.iloc[i, 3])
                 & (df['기준년월']==temp.iloc[i, 0])].index
        df.iloc[index, 3] = temp.iloc[i, 6]

        # Fill up other columns with 전월
        if temp.iloc[i, 1] != 201907:

            if temp.iloc[i, 1] == 201912:
                index = df[(df['종목번호']==temp.iloc[i, 3])
                        & (df['기준년월']==202001)].index
                df.iloc[index, 4:] = np.array(temp.iloc[i, 7:])

            else:
                index = df[(df['종목번호']==temp.iloc[i, 3])
                             & (df['기준년월']==temp.iloc[i, 0]+1)].index
                df.iloc[index, 4:] = np.array(temp.iloc[i, 7:])
    
    # Unify timestamp standard
    df['기준년월'] = pd.to_datetime(df['기준년월'], format='%Y%m')
    df['기준년월'] = df['기준년월'].dt.to_period('M')
    
    # Add stock factors    
    df = pd.merge(df, factor, how='left', on=['종목번호', '기준년월'])
    return df

In [4]:
for i in trange(1, 49):
    if i == 1:
        df = preprocess(i)
    else:
        df = pd.concat([df, preprocess(i)])

100%|██████████| 48/48 [03:32<00:00,  4.43s/it]


In [5]:
df['기준년월'] = df['기준년월'].apply(lambda x: str(x))

- 1-3에서 생성한 Exponential smoothing 결과 병합

In [6]:
factor_ES3 = pd.read_csv('data/stock_factor_ES3.csv')
factor_ES3 = factor_ES3.drop(columns=['매수고객수'])
factor_ES3

,기준년월,그룹번호,종목번호,매수고객수_ES3
0,2019-07,MAD01,A000660,0.000000
1,2019-08,MAD01,A000660,0.000000
2,2019-09,MAD01,A000660,0.000000
3,2019-10,MAD01,A000660,16.424354
4,2019-11,MAD01,A000660,16.075590
...,...,...,...,...
137561,2020-03,MAD48,A272210,0.000000
137562,2020-04,MAD48,A272210,0.000000
137563,2020-05,MAD48,A272210,0.000000
137564,2020-06,MAD48,A272210,0.000000


In [7]:
factor_ES6 = pd.read_csv('data/stock_factor_ES6.csv')
factor_ES6 = factor_ES6.drop(columns=['매수고객수'])
factor_ES6

,기준년월,그룹번호,종목번호,매수고객수_ES6
0,2019-07,MAD01,A000660,0.0
1,2019-08,MAD01,A000660,0.0
2,2019-09,MAD01,A000660,0.0
3,2019-10,MAD01,A000660,0.0
4,2019-11,MAD01,A000660,0.0
...,...,...,...,...
137561,2020-03,MAD48,A272210,0.0
137562,2020-04,MAD48,A272210,0.0
137563,2020-05,MAD48,A272210,0.0
137564,2020-06,MAD48,A272210,0.0


In [8]:
df = pd.merge(df, factor_ES3, how='left', on=['기준년월', '그룹번호', '종목번호'])
df = pd.merge(df, factor_ES6, how='left', on=['기준년월', '그룹번호', '종목번호'])
df.fillna(value=0, inplace=True)
df

,기준년월,그룹번호,종목번호,매수고객수,전월_매도고객수,전월_평균매수수량,전월_평균매도수량,전월_매수가격_중앙값,전월_매도가격_중앙값,종목명,...,고가/시가,저가/시가,시가종가 변화율,코로나19 영향,시가총액,외국인 지분율,순위,외국인 지분율 최대변동비율,매수고객수_ES3,매수고객수_ES6
0,2019-07,MAD01,A000020,0.0,0.0,0.0,0.0,0.0,0.0,동화약품,...,1.028926,0.898760,0.003099,0.0,273728406000,0.062922,567.043478,0.072488,0.0,0.0
1,2019-07,MAD01,A000080,0.0,0.0,0.0,0.0,0.0,0.0,하이트진로,...,1.104623,0.982968,-0.012165,0.0,1546446122550,0.106561,135.739130,0.138477,0.0,0.0
2,2019-07,MAD01,A000100,0.0,0.0,0.0,0.0,0.0,0.0,유한양행,...,1.032819,0.824324,-0.028958,0.0,3213444422500,0.235457,87.782609,0.021432,0.0,0.0
3,2019-07,MAD01,A000120,0.0,0.0,0.0,0.0,0.0,0.0,CJ대한통운,...,1.032967,0.945055,-0.003663,0.0,3205134332000,0.189239,84.434783,0.019670,0.0,0.0
4,2019-07,MAD01,A000140,0.0,0.0,0.0,0.0,0.0,0.0,하이트진로홀딩스,...,1.561810,0.983444,-0.006623,0.0,279641518250,0.058213,638.130435,0.177449,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678283,2020-07,MAD48,A237820,0.0,0.0,0.0,0.0,0.0,0.0,플레이디,...,1.078125,0.906250,-0.015625,1.0,"172,525,033,000",0.000604,423.782609,14.000000,0.0,0.0
678284,2020-07,MAD48,A198080,0.0,0.0,0.0,0.0,0.0,0.0,엔피디,...,1.077922,0.930736,-0.016234,1.0,"99,277,202,850",0.002587,665.217391,26.500000,0.0,0.0
678285,2020-07,MAD48,A353810,0.0,0.0,0.0,0.0,0.0,0.0,이지바이오,...,1.245747,0.901701,-0.054820,1.0,"134,459,606,400",0.045539,542.521739,0.427397,0.0,0.0
678286,2020-07,MAD48,A298060,0.0,0.0,0.0,0.0,0.0,0.0,에스씨엠생명과학,...,1.103825,0.779599,-0.100182,1.0,"329,625,395,550",0.015439,213.434783,inf,0.0,0.0


- 추가 데이터 병합

In [9]:
df_stock = pd.read_csv('data/stocks.csv', index_col=0)
df_stock = df_stock[df_stock['기준일자']==20200701]
df_stock = df_stock[['종목번호', '시장구분', '표준산업구분코드_대분류', '20년7월TOP3대상여부']]

In [10]:
df = pd.merge(df, df_stock, how='left', on=['종목번호'])

In [11]:
df['기준년월'] = df['기준년월'].apply(lambda x: int(x.replace('-', '')))
df['시가총액'] = df['시가총액'].apply(lambda x : int(x.replace(',', '')))

In [64]:
df.to_csv('data/data_preprocessed.csv', index=False)

# 2. 분석 모델 (1) : 추천 시스템 with item2vec

In [31]:
from gensim.models import Word2Vec

In [32]:
train = pd.read_csv("data/trade_train.csv")
stocks = pd.read_csv("data/stocks.csv")

In [33]:
tmp = train.groupby(['기준년월', '그룹번호', '종목번호'])['매수고객수'].sum().reset_index()
tmp = tmp[tmp['매수고객수']!=0]

## 2-1. 그룹별 매수 종목 sequence 생성

In [34]:
# Get stock sequence for each group

from collections import OrderedDict
g_num = tmp['그룹번호'].unique()
a = OrderedDict()

for i in g_num:
    a[i] = []

for i in tmp.iterrows():
    val = a[i[1][1]]
    val.append(i[1][2])
    a[i[1][1]] = val
    
index = list(a.keys())
# index.sort()
res = []
for i in index:
    res.append(a[i])

In [35]:
# Create a 'stock sequence per group' table

seq = pd.DataFrame({'그룹번호':tmp['그룹번호'].unique(), '시퀀스':res})
seq = seq.sort_values('그룹번호')

## 2-2. Word2vec 모델 학습

In [36]:
# Divide 'seq' data frame into 48 groups
# ex) seq0 : sequence table of MAD01

l = list(seq['그룹번호'])

for i in range(48):
    globals()['seq{}'.format(i)] = seq[seq['그룹번호']==l[i]]
    

# Word2vec

for i in range(48):
    globals()['model{}'.format(i)] = Word2Vec(globals()['seq{}'.format(i)]['시퀀스'], size=400, window = 1, min_count=0)


## 2-3. Matrix Multiplication

https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/

In [37]:
# Get word vectors from word2vec result
for i in range(48):
    globals()['word_vectors{}'.format(i)] = globals()['model{}'.format(i)].wv
    globals()['vocabs{}'.format(i)] = globals()['word_vectors{}'.format(i)].vocab.keys()
    globals()['word_vectors_list{}'.format(i)] = [globals()['word_vectors{}'.format(i)][v] for v in globals()['vocabs{}'.format(i)]]
    
    
# Create vectorm matrix
for i in range(48):
    globals()['vector{}'.format(i)] = pd.DataFrame({'종목명':list(globals()['vocabs{}'.format(i)]), '벡터':globals()['word_vectors_list{}'.format(i)]})
    
    
# Create cosine similarity matrix
from sklearn.metrics.pairwise import cosine_similarity

for i in range(48):
    globals()['similarity{}'.format(i)] = cosine_similarity(np.array(globals()['word_vectors_list{}'.format(i)]))
    globals()['similarity_matrix{}'.format(i)] = pd.DataFrame(data = globals()['similarity{}'.format(i)], index = globals()['vector{}'.format(i)]['종목명'], columns = globals()['vector{}'.format(i)]['종목명'])
    globals()['similarity_matrix{}'.format(i)].head()
    
    
# Create count matrix
for i in range(48):
    globals()['train{}'.format(i)] = train[train['그룹번호']==l[i]]
    
for i in range(48):
    globals()['count_matrix{}'.format(i)] = pd.pivot_table(globals()['train{}'.format(i)], index = '그룹번호', columns = '종목번호', values = '매수고객수', aggfunc = 'sum')
    globals()['count_matrix{}'.format(i)] = globals()['count_matrix{}'.format(i)].fillna(0)
    globals()['count_matrix{}'.format(i)] = globals()['count_matrix{}'.format(i)].astype(int)
    globals()['count_matrix{}'.format(i)] = globals()['count_matrix{}'.format(i)][globals()['similarity_matrix{}'.format(i)].columns]

In [38]:
# Matrix multiplication
# mul_matrix = count_matrix * similarity_matrix

for i in range(48):
    globals()['mul_matrix{}'.format(i)] = np.matmul(globals()['count_matrix{}'.format(i)], globals()['similarity_matrix{}'.format(i)])
    globals()['mul_matrix_T{}'.format(i)] = globals()['mul_matrix{}'.format(i)].T


In [39]:
# Select 202007 top3 target stocks

exp = stocks[stocks['20년7월TOP3대상여부']=='Y']
exp_list = exp['종목번호'].unique()

for i in range(48):
    t = []
    for j in range(len(globals()['mul_matrix_T{}'.format(i)])):
        if not globals()['mul_matrix_T{}'.format(i)].index[j] in exp_list:
            t.append(globals()['mul_matrix_T{}'.format(i)].index[j])
    globals()['mul_matrix_T{}'.format(i)].drop(t, inplace=True)

In [40]:
# Select top3 stocks of each group at 2020/07

for i in range(48):
    a = []
    b = []
    c = []
    for j in globals()['mul_matrix_T{}'.format(i)].columns:
        idx = list(globals()['mul_matrix_T{}'.format(i)].nlargest(3, [j]).index)
        idx.sort()
        a.append(idx[0])
        b.append(idx[1])
        c.append(idx[2])
        
    globals()['result{}'.format(i)] = pd.DataFrame({'그룹명':list(globals()['mul_matrix_T{}'.format(i)].columns), '종목번호1':a, '종목번호2':b, '종목번호3':c})
    
res_item2vec = pd.DataFrame(columns = result1.columns)
for i in range(48):
    res_item2vec = pd.concat([res_item2vec, globals()['result{}'.format(i)]])

# Save result
res_item2vec = res_item2vec.sort_values('그룹명')

## 2-4. 2020년 6월 예측 결과 실제와 비교

- 동일 모델로 2020년 6월 예측

In [41]:
tmp = train.groupby(['기준년월', '그룹번호', '종목번호'])['매수고객수'].sum().reset_index()
tmp = tmp[tmp['매수고객수']!=0]
tmp = tmp[tmp['기준년월']!=202006]

In [42]:
from collections import OrderedDict
g_num = tmp['그룹번호'].unique()
a = OrderedDict()

for i in g_num:
    a[i] = []
    
for i in tmp.iterrows():
    val = a[i[1][1]]
    val.append(i[1][2])
    a[i[1][1]] = val
    
index = list(a.keys())
# index.sort()
res = []
for i in index:
    res.append(a[i])

In [43]:
seq = pd.DataFrame({'그룹번호':tmp['그룹번호'].unique(), '시퀀스':res})
seq = seq.sort_values('그룹번호')

In [44]:
l = list(seq['그룹번호'])

for i in range(48):
    globals()['seq{}'.format(i)] = seq[seq['그룹번호']==l[i]]


In [45]:
for i in range(48):
    globals()['model{}'.format(i)] = Word2Vec(globals()['seq{}'.format(i)]['시퀀스'], size=400, window = 1, min_count=0)

In [46]:
# Get results from word2vec
for i in range(48):
    globals()['word_vectors{}'.format(i)] = globals()['model{}'.format(i)].wv
    globals()['vocabs{}'.format(i)] = globals()['word_vectors{}'.format(i)].vocab.keys()
    globals()['word_vectors_list{}'.format(i)] = [globals()['word_vectors{}'.format(i)][v] for v in globals()['vocabs{}'.format(i)]]
    
    
# Create word vector matrix
for i in range(48):
    globals()['vector{}'.format(i)] = pd.DataFrame({'종목명':list(globals()['vocabs{}'.format(i)]), '벡터':globals()['word_vectors_list{}'.format(i)]})
    
    
# Create cosine similarity matrix
from sklearn.metrics.pairwise import cosine_similarity

for i in range(48):
    globals()['similarity{}'.format(i)] = cosine_similarity(np.array(globals()['word_vectors_list{}'.format(i)]))
    globals()['similarity_matrix{}'.format(i)] = pd.DataFrame(data = globals()['similarity{}'.format(i)], index = globals()['vector{}'.format(i)]['종목명'], columns = globals()['vector{}'.format(i)]['종목명'])
    globals()['similarity_matrix{}'.format(i)].head()
    

# Create count matrix
for i in range(48):
    globals()['train{}'.format(i)] = train[train['그룹번호']==l[i]]
    
for i in range(48):
    globals()['count_matrix{}'.format(i)] = pd.pivot_table(globals()['train{}'.format(i)], index = '그룹번호', columns = '종목번호', values = '매수고객수', aggfunc = 'sum')
    globals()['count_matrix{}'.format(i)] = globals()['count_matrix{}'.format(i)].fillna(0)
    globals()['count_matrix{}'.format(i)] = globals()['count_matrix{}'.format(i)].astype(int)
    globals()['count_matrix{}'.format(i)] = globals()['count_matrix{}'.format(i)][globals()['similarity_matrix{}'.format(i)].columns]

In [47]:
for i in range(48):
    globals()['mul_matrix{}'.format(i)] = np.matmul(globals()['count_matrix{}'.format(i)], globals()['similarity_matrix{}'.format(i)])
    globals()['mul_matrix_T{}'.format(i)] = globals()['mul_matrix{}'.format(i)].T

for i in range(48):
    a = []
    b = []
    c = []
    for j in globals()['mul_matrix_T{}'.format(i)].columns:
        idx = list(globals()['mul_matrix_T{}'.format(i)].nlargest(3, [j]).index)
        idx.sort()
        a.append(idx[0])
        b.append(idx[1])
        c.append(idx[2])
        
    globals()['result{}'.format(i)] = pd.DataFrame({'그룹명':list(globals()['mul_matrix_T{}'.format(i)].columns), '종목번호1':a, '종목번호2':b, '종목번호3':c})
    
res_item2vec_202006 = pd.DataFrame(columns = result1.columns)
for i in range(48):
    res_item2vec_202006 = pd.concat([res_item2vec_202006, globals()['result{}'.format(i)]])
    
res_item2vec_202006 = res_item2vec_202006.sort_values('그룹명')

- 예측결과 평가

In [68]:
#데이터 불러오기
df = pd.read_csv('data/data_preprocessed_200927.csv',index_col='기준년월')
stock=pd.read_csv('data/stocks.csv')

In [69]:
# Top3종목 알기 위한 함수들

def dataPreprocessed(tradeData,groupName):
    data=tradeData[(tradeData['그룹번호']==groupName)]
    data=data.dropna(axis=0)
    
    return data
def dataSplit(tradeData,groupName,month):
    data=dataPreprocessed(tradeData,groupName)
    data_val=data[data.index==month] 
    #Validation data
    y_val=data_val['매수고객수']
    
    y_val=np.array(y_val)
    
    return y_val


def predict(y_val):
    

    #원래 값들 정렬
    valdf=pd.DataFrame(y_val)
    sorted_valdf=valdf.sort_values(by=0,ascending=False).head(30)
    #list에 valdf iloc index 정렬
    valdf_index_list=[]
    for k in range(len(sorted_valdf)):
        valdf_index_list.append(sorted_valdf.iloc[k].name)
    
    return valdf_index_list

def comp_predNreal(tradeData,groupName,stock_data,real_list):
    data=dataPreprocessed(tradeData,groupName)
       
    #실제 코드, 이름 저장할 리스트
    real_stockCode=[]
    real_stockName=[]
    
    for k in real_list:
        code=data.iloc[k]['종목번호']
        if (code in real_stockCode):
            pass
        else:
            real_stockCode.append(code)
            real_stockName.append(stock_data[stock_data['종목번호']==code].iloc[0]['종목명'])
    real_stock_Code_Top3=real_stockCode[0:3]
    
    return real_stockCode,real_stockName

In [70]:
# main code, 알고 싶은 month를 설정해 주고 코드를 돌리면 됩니다
allcode=[]
allname=[]
for k in range(1,49):
    name='MAD'+str(k).zfill(2) 
    y_val=dataSplit(df,name,202006)  #이부분에 202006, 202005 등등으로 원하는 월을 바꾸면 됩니다.
    a=predict(y_val)
    c,d=comp_predNreal(df,name,stock,a)
    code=c[0:3]
    name=d[0:3]
    allcode.append(code)
    allname.append(name)
#딕셔너리 a는 종목코드
#딕셔너리 b는 종목명
a={}
b={}
for k in range (0,48):
    name='MAD'+str(k+1).zfill(2)
    a[name]=allcode[k]
    b[name]=allname[k]
    
#위에서 만든 딕셔너리를 answer_sheet형식으로 변환하기

def codeconvert(dictionary):
    ans=pd.DataFrame(dictionary).transpose()
    ans.columns = ['종목번호1', '종목번호2', '종목번호3']
    ans=ans.reset_index()
    ans.rename(columns = {'index' : '그룹명'}, inplace = True)
    final=ans.set_index('그룹명')
    return final

fin=codeconvert(a)
fin

,종목번호1,종목번호2,종목번호3
그룹명,,,
MAD01,A005930,A000660,A005935
MAD02,A005930,A104040,A054780
MAD03,A003000,A018880,A007570
MAD04,A005930,A000660,A003000
MAD05,A035620,A035250,A067990
MAD06,A005930,A035620,A035250
MAD07,A005930,A096630,A010950
MAD08,A018880,A003000,A007570
MAD09,A035620,A005930,A051780


- 예측한 값과 실제 값 비교

In [56]:
#실제로 Top3인것:real
#w2v으로 예측한 Top3인것:pred

real=fin
pred=res_item2vec_202006

In [57]:
#맞춘 개수 파악
correct_num=[]
for k in range(0,48):
    list1=[]
    temp_real_list=list(real.iloc[k])
    list2=[]
    temp_pred_list=list(pred.iloc[k])
    c=0
    for p in temp_pred_list:
        if p in temp_real_list:
            c+=1
    correct_num.append(c)

#맞춘 개수 집단별로 구분해서 데이터프레임으로 나타내기
group=[]
for k in range (0,48):
    name='MAD'+str(k+1).zfill(2)
    group.append(name)
    
a={}
a['name']=group
a['score']=correct_num
result=pd.DataFrame(a)
result
#결과물로 w2v이 예측한 Top3와 실제Top3이 일치하는 개수가 집단별로 산정되어 나옵니다.

,name,score
0,MAD01,3
1,MAD02,1
2,MAD03,0
3,MAD04,2
4,MAD05,0
5,MAD06,1
6,MAD07,1
7,MAD08,0
8,MAD09,1
9,MAD10,0


---
#### 예측력이 낮은 8, 12, 20, 32, 41, 42번 group에 대해 다른 분석방법 적용
---

# 3. 분석 모델 (2): LGBM Regressor with Bayesian Opimization

In [ ]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTENC, SMOTE
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from functools import partial
from sklearn.metrics import accuracy_score
import warnings        
warnings.filterwarnings("ignore")

In [8]:
df_raw = pd.read_csv('data/data_preprocessed.csv')

### 3-1. 분석 전 data preprocess

In [9]:
# Drop useless or flawed columns
df = df_raw.drop(columns=['종목명', '종목번호', '20년7월TOP3대상여부', '외국인 지분율 최대변동비율', '전월대비_외국인 지분율 최대변동비율'])

# Convert data type to category
df['시장구분'] = df['시장구분'].astype('category')
df['표준산업구분코드_대분류'] = df['표준산업구분코드_대분류'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678288 entries, 0 to 678287
Data columns (total 38 columns):
기준년월            678288 non-null int64
그룹번호            678288 non-null object
매수고객수           678288 non-null float64
전월_매도고객수        678288 non-null float64
전월_평균매수수량       678288 non-null float64
전월_평균매도수량       678288 non-null float64
전월_매수가격_중앙값     678288 non-null float64
전월_매도가격_중앙값     678288 non-null float64
거래량             678288 non-null float64
종목고가            678288 non-null float64
종목저가            678288 non-null float64
최고증가율           678288 non-null float64
최저증가율           678288 non-null float64
평균증가율           678288 non-null float64
당일등락량           678288 non-null float64
거래중지이력          678288 non-null float64
고가/시가           678288 non-null float64
저가/시가           678288 non-null float64
시가종가 변화율        678288 non-null float64
코로나19 영향        678288 non-null float64
시가총액            678288 non-null int64
외국인 지분율         678288 non-null float64
순위            

In [12]:
# Scale data
scaler = StandardScaler()
col_num = df.columns.get_loc('시장구분')
df.iloc[:, 3:col_num] = scaler.fit_transform(df.iloc[:, 3:col_num])
df.iloc[:, col_num+2:] = scaler.fit_transform(df.iloc[:, col_num+2:])

In [15]:
# Split train test data
x_train = df.drop(columns=['매수고객수'])
x_test = x_train[x_train['기준년월']==202007]
x_train = x_train[x_train['기준년월']!=202007]
x_train.reset_index(inplace=True, drop=True)
x_test.reset_index(inplace=True, drop=True)

y_train = df[['기준년월', '매수고객수']]
y_test = y_train[y_train['기준년월']==202007]['매수고객수']
y_train = y_train[y_train['기준년월']!=202007]['매수고객수']
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

## 3-2. 분석 모델 적용

In [16]:
# Accuracy metrics for evaluate output accuracy
def accuracy(true, pred):
    if len(true)!=len(pred):
        raise Exception('Length of true and prediction values are not equal')
        sys.exit()
    
    correct = 0
    for i in range(len(true)):
        if pred[i] in true:
            correct += 1
    
    return correct/len(true)

In [18]:
def lgb_reg(num_leaves, 
           learning_rate, 
           n_estimators, 
           #subsample, 
           #colsample_bytree, 
           reg_alpha, 
           reg_lambda,
           #subsample_freq, 
           min_child_samples, 
           subsample_for_bin, 
           min_child_weight,
           x_data=None, y_data=None, output='score', n_splits=1, group=1):

    
    score = 0
    #smoter = SMOTENC(categorical_features=[x_data.columns.get_loc('시장구분'), x_data.columns.get_loc('표준산업구분코드_대분류')], random_state=42)
    #smoter = SMOTE(random_state=42)
    models = []

    group_index = x_data[x_data['그룹번호']=='MAD'+str(group).zfill(2)].index
    x_data, y_data = x_data.iloc[group_index].drop(columns=['그룹번호']), y_data.iloc[group_index]
    x_data.reset_index(inplace=True, drop=True)
    y_data.reset_index(inplace=True, drop=True)

    for valid_ym in range(202007-n_splits, 202007):
        # Split train and valid data
        train_index = x_data[x_data['기준년월']<valid_ym].index
        valid_index = x_data[x_data['기준년월']>=valid_ym].index
        x_train, y_train = x_data.iloc[train_index].drop(columns=['기준년월']), y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index].drop(columns=['기준년월']), y_data.iloc[valid_index]
        #x_train, y_train = smoter.fit_resample(x_train, y_train)

        # Define LGBM Classifier model parameters
        model = LGBMRegressor(
        num_leaves = int(num_leaves), 
        learning_rate = learning_rate, 
        n_estimators = int(n_estimators), 
        #subsample = np.clip(subsample, 0, 1), 
        #colsample_bytree = np.clip(colsample_bytree, 0, 1), 
        reg_alpha = reg_alpha, 
        reg_lambda = reg_lambda,
        boosting_type = 'gbdt',
        #subsample_freq = int(subsample_freq),
        min_child_samples = int(min_child_samples),
        subsample_for_bin = int(subsample_for_bin),
        min_child_weight = min_child_weight,
        categorical_feature = [x_train.columns.get_loc('시장구분'), x_train.columns.get_loc('표준산업구분코드_대분류')],
        n_jobs=-1
        )
        
        model.fit(x_train, y_train)
        models.append(model)

        pred = model.predict(x_valid)
        score -= mean_squared_error(y_valid, pred)/n_splits
        
        result = []
        for i in range(3):
            result.append(df_raw.loc[pred_index[i], '종목번호'])
        
    if output == 'score':
        return score
    if output == 'model':
        return models
    if output == 'result':
        return result

In [35]:
# Evaluate result and return result as a form of submission file
def predict_value(group, model, test):
    
    test = test[test['그룹번호']=='MAD'+str(group).zfill(2)].drop(columns=['기준년월', '그룹번호'])
    pred = model.predict(test)
    pred_index = list(pd.DataFrame(pred).sort_values(0, ascending=False).head(15).index)
    
    result = []
    for i in range(15):
        company_code = df_raw.loc[pred_index[i], '종목번호']
        if df_raw.loc[pred_index[i], '20년7월TOP3대상여부'] == 'Y':
            result.append(df_raw.loc[pred_index[i], '종목번호'])
            result.append(pred[i])
        if len(result) == 3:
            break
        
    return result

In [37]:
group_list = []
for i in range(1,49):
    group_list.append('MAD'+str(i).zfill(2))
submission = pd.DataFrame(group_list, columns=['그룹명'])
submission['종목번호1'] = 0
submission['종목번호2'] = 0
submission['종목번호3'] = 0

models = []
feature_imp = []
valid_score = 0
for i in range(1,49):
    print(f'GROUP {i}')
    func_fixed = partial(lgb_reg, x_data=x_train, y_data=y_train, output='score', n_splits=4, group=i)
    lgbBO = BayesianOptimization(
        func_fixed, 
        {
            'num_leaves': (256, 2048),
            'learning_rate': (0.001, 0.1),
            'n_estimators': (256, 2048),
            #'subsample': (0, 1),
            #'colsample_bytree': (0, 1),
            'reg_alpha': (0, 20),
            'reg_lambda': (0, 100),
            #'subsample_freq': (0, 20),
            'min_child_samples': (1, 64),
            'subsample_for_bin': (20000, 30000),
            'min_child_weight': (0, 50)
        },
        random_state=4321
    )
    lgbBO.maximize(init_points=5, n_iter=15)
    
    params = lgbBO.max['params']
    valid_score += lgbBO.max['target']/48
    models = lgb_reg(
        params['num_leaves'], 
        params['learning_rate'], 
        params['n_estimators'], 
        #params['subsample'], 
        #params['colsample_bytree'], 
        params['reg_alpha'], 
        params['reg_lambda'],
        #params['subsample_freq'], 
        params['min_child_samples'], 
        params['subsample_for_bin'],
        params['min_child_weight'],
        x_data=x_train, y_data=y_train, output='model', n_splits=4, group=i)
    
    feature_imp.append(models[0].feature_importances_)        
    submission.iloc[i-1, 1:4] = predict_value(i, models[0], x_test)

print(f'Average valid score: {valid_score}')

GROUP 1
|   iter    |  target   | learni... | min_ch... | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsam... |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -8.206    |  0.008009 |  52.35    |  38.4     |  769.1    |  602.0    |  19.58    |  40.62    |  2.758e+0 |
|  2        | -7.363    |  0.009826 |  20.52    |  30.95    |  1.08e+03 |  647.2    |  13.27    |  67.87    |  2.95e+04 |
|  3        | -6.778    |  0.02884  |  40.05    |  19.16    |  973.4    |  1.945e+0 |  18.6     |  94.84    |  2.375e+0 |
|  4        | -6.811    |  0.03489  |  42.88    |  2.116    |  672.2    |  1.027e+0 |  1.559    |  76.66    |  2.854e+0 |
|  5        | -8.28     |  0.01589  |  7.372    |  13.55    |  310.1    |  1.757e+0 |  11.95    |  93.11    |  2.496e+0 |
|  6        | -4.831    |  0.06901  |  42.42    |  6.547    |  1.161e+0 |  1.48e+03 |  10.89    |  29.54    |  2.708e+0 |
|  7        | -4

In [38]:
submission.to_csv('data/submission_LGBM.csv', index=False)

## 4. 모델 결합 및 최종 결과 반환

In [ ]:
# Create empty dataframe
res_bayes_word2vec = pd.DataFrame(columns = bayes_group.columns)
bayes_group = pd.read_csv('data/submission_LGBM.csv')
word2vec_group = res_item2vec

# List up groups which have to use LGBM model
bayes_num = [8, 12, 20, 32, 41, 42]

# word2vec이랑 bayes 결과 합치기
for i in range(48):
    if i+1 in bayes_num:
        res_bayes_word2vec = pd.concat([res_bayes_word2vec, bayes_group[bayes_group.index==i]])
    else:
        res_bayes_word2vec = pd.concat([res_bayes_word2vec, word2vec_group[word2vec_group.index ==i]])
        
res_elastic_word2vec.to_csv('final_submission.csv')